In [1]:
#libs for scraper
from bs4 import BeautifulSoup
import requests
import pandas as pd
import lxml

#get data from page, clean for export
url = 'https://www.formula1.com/en/results.html/2022/races/1124/bahrain/qualifying.html'

# Request content from web page
result = requests.get(url)
c = result.content

# Set as Beautiful Soup Object
soup = BeautifulSoup(c,'html.parser')

tbl= soup.find("table",{'class':'resultsarchive-table'})

data_frame = pd.read_html(str(tbl))[0]

data_frame = data_frame.drop(data_frame.columns[0], axis=1)
data_frame = data_frame.drop(data_frame.columns[8], axis=1)

colors = pd.read_csv('f1_colors.csv')

data_frame = pd.merge(data_frame,colors,how='left',on='Car')
data_frame

,Pos,No,Driver,Car,Q1,Q2,Q3,Laps,Color
0,1,16,Charles Leclerc LEC,Ferrari,1:31.471,1:30.932,1:30.558,15,#DC0000
1,2,1,Max Verstappen VER,Red Bull Racing RBPT,1:31.785,1:30.757,1:30.681,14,#0600EF
2,3,55,Carlos Sainz SAI,Ferrari,1:31.567,1:30.787,1:30.687,15,#DC0000
3,4,11,Sergio Perez PER,Red Bull Racing RBPT,1:32.311,1:31.008,1:30.921,18,#0600EF
4,5,44,Lewis Hamilton HAM,Mercedes,1:32.285,1:31.048,1:31.238,17,#00D2BE
5,6,77,Valtteri Bottas BOT,Alfa Romeo Ferrari,1:31.919,1:31.717,1:31.560,15,#900000
6,7,20,Kevin Magnussen MAG,Haas Ferrari,1:31.955,1:31.461,1:31.808,12,#FFFFFF
7,8,14,Fernando Alonso ALO,Alpine Renault,1:32.346,1:31.621,1:32.195,14,#0090FF
8,9,63,George Russell RUS,Mercedes,1:32.269,1:31.252,1:32.216,17,#00D2BE
9,10,10,Pierre Gasly GAS,AlphaTauri RBPT,1:32.096,1:31.635,1:32.338,21,NaN


In [3]:
#libs for dashboard
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash import Dash
from dash.dependencies import Input, Output
import plotly.graph_objects as go

# Load Data
df = data_frame
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("F1 - Bahrain Qualifier 2022"),
    html.Label([
        "Select a driver",
        dcc.Dropdown(
            id='driver-dropdown', clearable=False,
            value='Max Verstappen VER', options=df['Driver'])
        ]),
    html.Br(),
    dcc.Graph(id='bar'),

])


@app.callback(
    Output(component_id='bar', component_property='figure'),
    [Input(component_id='driver-dropdown', component_property='value')]
)

# utility build display string from nanoseconds


def update_figure(input_value):
    #input and filtered df
    filtered_df = df[df.Driver == input_value]
    df_for_fig = pd.DataFrame({'Race':['Q1','Q2','Q3'],'Time':[filtered_df['Q1'].iloc[0],filtered_df['Q2'].iloc[0],filtered_df['Q3'].iloc[0]]})
    df_for_fig = df_for_fig.dropna()
    
    # convert to timedelta...
    df_for_fig['Time'] = (
        df_for_fig['Time']
        .str.extract(r"(?P<minute>[0-9]+):(?P<sec>[0-9]+).(?P<milli>[0-9]+)")
        .apply(
            lambda r: pd.Timestamp(year=1970,month=1,day=1,
                                   minute=int(r.minute),second=int(r.sec),microsecond=int(r.milli) * 10 ** 3,
            ),
            axis=1,
        )
        - pd.to_datetime("1-jan-1970").replace(hour=0, minute=0, second=0, microsecond=0)
        )

    #normalize string text
    def strfdelta(t, fmt="{minutes:02d}:{seconds:02d}.{milli:03d}"):
        d = {}
        d["minutes"], rem = divmod(t, 10 ** 9 * 60)
        d["seconds"], d["milli"] = divmod(rem, 10 ** 9)
        d["milli"] = d["milli"] // 10**6
        return fmt.format(**d)
    
    color = filtered_df['Color'].iloc[0]
    #figure
    #fig = px.bar(df_for_fig,x='Race',y='Time',hover_name=df_for_fig["Time"].astype('int64').apply(strfdelta),color_continuous_scale=color)
    fig  = go.Figure(
        data=go.Bar(x=df_for_fig['Race'],y=df_for_fig['Time'],
                        marker_color=color))
    # fix up tick labels
    ticks = pd.Series(range(df_for_fig['Time'].astype('int64').min() - 10 ** 10,df_for_fig['Time'].astype('int64').max(),10 ** 10,))
    fig.update_layout(
        xaxis={
            "range": [
                df_for_fig[input_value].astype('int64').min(),
                df_for_fig[input_value].astype('int64').max(),
            ],
            "tickmode": "array",
            "tickvals": ticks,
            "ticktext": ticks.apply(strfdelta),
            "side":"top",
            "showgrid":False
        },
    yaxis={'categoryorder':'total descending',"showgrid":False},
    autosize=False,
    width=600,
    height= 600,
    plot_bgcolor ='rgba(0, 0, 0, 0)',
    paper_bgcolor = 'rgba(0, 0, 0, 0)')
    
    return fig


if __name__ == '__main__':
    app.run_server(debug=True,port=8002,mode='inline')